In [1]:
import transformers
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

/home/zhouhang/miniconda3/envs/transformers/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
base_model_name_or_path = './checkpoints/generative_movie_title_ner/checkpoint-900'
tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path)
model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name_or_path)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.94s/it]


### Basic Usage

In [131]:
prompt = """Perform entity recognition and extract movie titles from the following text: \n"""
text = """who directed the pirates of the carribean movie"""

In [132]:
import torch
with torch.no_grad():
    inputs = tokenizer(prompt+text.replace('\n',''), return_tensors="pt")
    outputs = model.generate(**inputs)
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

/home/zhouhang/miniconda3/envs/transformers/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['Pirates of the Caribbean']


### Batched Usage on GPU

In [137]:
import torch
device = torch.device('cuda')
model = model.to(device)
texts = ["""who directed the pirates of the carribean movie""", "I love Police Academy 2, too!", 'some random stuff...']

In [138]:
encoding = tokenizer(texts, padding=True, return_tensors='pt').to(device)
with torch.no_grad():
    generated_ids = model.generate(**encoding)
generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
generated_texts

['James Cameron', 'Police Academy 2', 'None']